## (Updated) Problem 2: K-nearest neighbor (KNN) for binary classification

#### Some notes

In this task, we will use three distance functions: (we removed the vector symbol for simplicity)

- Euclidean distance:  $$d(x, y) = \sqrt{\langle x - y, x - y \rangle}$$
- Inner product distance: $$d(x, y ) = \langle x, y \rangle$$
- Gaussian kernel distance: 
    $$d(x, y ) = - \exp({−\frac 12 || x - y ||^2}) $$


F1-score is a important metric for binary classification, as sometimes the accuracy metric has the false positive (a good example is in MLAPP book 2.2.3.1 “Example: medical diagnosis”, Page 29).

### Part 2.1 Distance Functions

Implement the class in file *project1_knn.py*
    - KNN
    
and the functions in *utils.py*    
    - f1_score
    - euclidean_distance
    - inner_product_distance
    - gaussian_kernel_distance

In [3]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from project1_knn import KNN
from utils import euclidean_distance, gaussian_kernel_distance, inner_product_distance
from utils import f1_score

distance_funcs = {
    'euclidean': euclidean_distance,
    'gaussian': gaussian_kernel_distance,
    'inner_prod': inner_product_distance,
}

#### Data processing 

Do the following steps:

- Load data (features and values) from function generate data cancer
- Check that there are 569 data samples and each sample have a feature vector of length 30.
- Split the whole data set into three parts:
     - the train set contains first 400 samples (0th - 399th samples),
     - the validation set contains the next 60 samples (400th - 459th samples),
     - the test set contains the rest 109 samples (460th - 568th samples).

In [4]:
from data import generate_data_cancer
features, labels = generate_data_cancer()

train_features, train_labels = features[:400], labels[:400]
valid_features, valid_labels = features[400:460], labels[400:460]
test_features, test_labels = features[460:], labels[460:]

assert len(train_features) == len(train_labels) == 400
assert len(valid_features) == len(valid_labels) == 60
assert len(test_features) == len(test_labels) == 109

#### Model selection 
In kNN model, the parameter k is a hyper-parameter. In this task, we only search k among {1, 3, 10, 20, 50}.

In [5]:
for name, func in distance_funcs.items():
    best_f1_score, best_k = -1, 0
    for k in [1, 3, 10, 20, 50]:
        model = KNN(k=k, distance_function=func)
        model.train(train_features, train_labels)
        train_f1_score = f1_score(
            train_labels, model.predict(train_features))

        valid_f1_score = f1_score(
            valid_labels, model.predict(valid_features))
        print('[part 2.1] {name}\tk: {k:d}\t'.format(name=name, k=k) + 
              'train: {train_f1_score:.5f}\t'.format(train_f1_score=train_f1_score) +
              'valid: {valid_f1_score:.5f}'.format(valid_f1_score=valid_f1_score))

        if valid_f1_score > best_f1_score:
            best_f1_score, best_k = valid_f1_score, k

    model = KNN(k=best_k, distance_function=func)
    model.train(train_features + valid_features,
                train_labels + valid_labels)
    test_f1_score = f1_score(test_labels, model.predict(test_features))
    print()
    print('[part 2.1] {name}\tbest_k: {best_k:d}\t'.format(name=name, best_k=best_k) +
          'test f1 score: {test_f1_score:.5f}'.format(test_f1_score=test_f1_score))
    print()

[part 2.1] euclidean	k: 1	train: 1.00000	valid: 0.96774
[part 2.1] euclidean	k: 3	train: 0.95879	valid: 0.97872
[part 2.1] euclidean	k: 10	train: 0.95896	valid: 0.97872
[part 2.1] euclidean	k: 20	train: 0.95054	valid: 0.97872
[part 2.1] euclidean	k: 50	train: 0.92405	valid: 0.96774

[part 2.1] euclidean	best_k: 3	test f1 score: 0.95000

[part 2.1] gaussian	k: 1	train: 1.00000	valid: 0.97826
[part 2.1] gaussian	k: 3	train: 0.96760	valid: 0.97826
[part 2.1] gaussian	k: 10	train: 0.96552	valid: 0.97826
[part 2.1] gaussian	k: 20	train: 0.94515	valid: 0.97826
[part 2.1] gaussian	k: 50	train: 0.93277	valid: 0.97872

[part 2.1] gaussian	best_k: 50	test f1 score: 0.96933

[part 2.1] inner_prod	k: 1	train: 0.72408	valid: 0.87850
[part 2.1] inner_prod	k: 3	train: 0.72408	valid: 0.87850
[part 2.1] inner_prod	k: 10	train: 0.72408	valid: 0.87850
[part 2.1] inner_prod	k: 20	train: 0.72408	valid: 0.87850
[part 2.1] inner_prod	k: 50	train: 0.72408	valid: 0.87850

[part 2.1] inner_prod	best_k: 1	test f

### Part 2.2 Data transformation

We are going to add one more step (data transformation) in the data processing part and see how it works. 
Sometimes, normalization plays an important role to make a machine learning model work (check term “Feature scaling” in wiki).

Here, we take two different data transformation approaches.

#### Normalizing the feature vector 

This one is simple but some times may work well. Given a feature vector $x$, the normalized feature vector is given by 

$$ x' = \frac x {\sqrt{\langle x, x \rangle}} $$
If a vector is a all-zero vector, we let the normalized vector also be a all-zero vector.


#### Min-max scaling the feature matrix

The above normalization is data independent, that is to say, the output of the normalization function doesn’t depend on the rest training data. However, sometimes it would be helpful to do data dependent normalization. One thing to note is that, when doing data dependent normalization, we can only use training data, as the test data is assumed to be unknown during training (at least for most classification tasks).

The min-max scaling works as follows: after min-max scaling, all values of training data’s feature vectors are in the given range.
Note that this doesn’t mean the values of the validation/test data’s fea- tures are all in that range, because the validation/test data may have dif- ferent distribution as the training data.

> Reference: https://en.wikipedia.org/wiki/Feature_scaling

> For short, you should keep the global max and min in the training set

**Implement** the classes in *utils.py*    
    - NormalizationScaler
    - MinMaxScaler

In [6]:
from utils import NormalizationScaler, MinMaxScaler

scaling_functions = {
    'min_max_scale': MinMaxScaler,
    'normalize': NormalizationScaler,
}

#### Model selection

Repeat the model selection part in part 2.2.

In [12]:
for scaling_name, scaling_class in scaling_functions.items():
    for name, func in distance_funcs.items():
        scaler = scaling_class()
        train_features_scaled = scaler(train_features)
        #print(train_features_scaled)
        valid_features_scaled = scaler(valid_features)

        best_f1_score, best_k = 0, -1
        for k in [1, 3, 10, 20, 50]:
            model = KNN(k=k, distance_function=func)
            model.train(train_features_scaled, train_labels)
            train_f1_score = f1_score(
                train_labels, model.predict(train_features_scaled))
            
            valid_f1_score = f1_score(
                valid_labels, model.predict(valid_features_scaled))
            #print('[part 2.2] {name}\t{scaling_name}\tk: {k:d}\t'.format(name=name, scaling_name=scaling_name, k=k) +
                  #'train: {train_f1_score:.5f}\t'.format(train_f1_score=train_f1_score) + 
                  #'valid: {valid_f1_score:.5f}'.format(valid_f1_score=valid_f1_score))

            if valid_f1_score > best_f1_score:
                best_f1_score, best_k = valid_f1_score, k
    

        # now change it to new scaler, since the training set changes
        scaler = scaling_class()
        combined_features_scaled = scaler(train_features + valid_features)
        test_features_scaled = scaler(test_features)

        model = KNN(k=best_k, distance_function=func)
        model.train(combined_features_scaled, train_labels + valid_labels)
        test_f1_score = f1_score(test_labels, model.predict(test_features_scaled))
        print()
        print('[part 2.2] {name}\t{scaling_name}\t'.format(name=name, scaling_name=scaling_name) +
              'best_k: {best_k:d}\ttest: {test_f1_score:.5f}'.format(best_k=best_k, test_f1_score=test_f1_score))
        print()

[[0.521037, 0.022658, 0.545989, 0.364042, 0.68001, 0.792037, 0.70314, 0.731113, 0.668446, 0.605518, 0.356147, 0.120469, 0.369034, 0.281983, 0.159296, 0.351398, 0.135682, 0.300625, 0.311645, 0.183042, 0.69246, 0.141525, 0.750126, 0.564802, 0.601136, 0.619292, 0.56861, 0.912027, 0.598462, 0.418864], [0.643144, 0.272574, 0.615783, 0.502017, 0.270471, 0.181768, 0.203608, 0.348757, 0.344367, 0.141323, 0.156437, 0.082589, 0.12444, 0.128965, 0.119387, 0.081323, 0.04697, 0.253836, 0.084539, 0.09111, 0.676984, 0.303571, 0.605903, 0.545399, 0.347553, 0.154563, 0.192971, 0.639175, 0.23359, 0.222878], [0.601496, 0.39026, 0.595743, 0.449798, 0.572941, 0.431017, 0.462512, 0.635686, 0.48158, 0.211247, 0.229622, 0.094303, 0.18037, 0.167451, 0.150831, 0.283955, 0.096768, 0.389847, 0.20569, 0.127006, 0.620635, 0.360075, 0.570686, 0.469324, 0.48359, 0.385375, 0.359744, 0.835052, 0.403706, 0.213433], [0.21009, 0.360839, 0.233501, 0.102993, 0.973233, 0.811361, 0.565604, 0.522863, 0.763481, 1.0, 0.139091, 0

TypeError: 'NoneType' object is not iterable